In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
#分析上证指数 数据

In [30]:
# df=pd.read_csv('D:/Program Files/收藏/笔记/量化投资/上证指数000001.csv',encoding='gbk',parse_dates=['candle_end_time'])
df=pd.read_csv('E:/Personal/中证白酒指数399997.csv',encoding='gbk',parse_dates=['日期'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 12 columns):
日期      2864 non-null datetime64[ns]
股票代码    2864 non-null object
名称      2864 non-null object
收盘价     2864 non-null float64
最高价     2864 non-null float64
最低价     2864 non-null float64
开盘价     2864 non-null float64
前收盘     2864 non-null object
涨跌额     2864 non-null object
涨跌幅     2864 non-null object
成交量     2864 non-null int64
成交金额    2864 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 268.6+ KB


In [ ]:
df['ndate']=pd.to_datetime(df.date)
df['month']=df.ndate.apply(lambda x:x.month)
df.head()
# 日期	股票代码	名称	收盘价	最高价	最低价	开盘价	前收盘	涨跌额	涨跌幅	换手率	成交量	成交金额	总市值	流通市值

In [ ]:
# df_1=df[df.month ==6][df.ndate>'2019-01-01']
df_1=df
fig,axl=plt.subplots(figsize=(20,20))
mpf.candlestick2_ohlc(axl,df_1.openingPrice.values,df_1.topPrice.values,df_1.bottomPrice.values,df_1.closingPrice.values,
                      width=1,colorup='b',colordown='r')
#显示均线
df['ma5']=df.openingPrice.rolling(window=5).mean()
df['ma30']=df.openingPrice.rolling(window=30).mean()
l=[i for i in range(df_1.date.count())]


In [ ]:
plt.plot(df.date.values,df.closingPrice.values,color='red',linewidth=2.0,linestyle='--')
plt.plot(df.date.values,df.openingPrice.values,color='blue',linewidth=3.0,linestyle='-.')
plt.show()

In [10]:
da=np.asarray(df)
da

array([['2020/2/10', "'000001", '上证指数', ..., 2, nan, nan],
       ['2020/2/7', "'000001", '上证指数', ..., 2, nan, nan],
       ['2020/2/6', "'000001", '上证指数', ..., 2, nan, nan],
       ...,
       ['1990/12/21', "'000001", '上证指数', ..., 12, 118.6540000000062,
        129.59333333333862],
       ['1990/12/20', "'000001", '上证指数', ..., 12, 114.4600000000062,
        128.75333333333865],
       ['1990/12/19', "'000001", '上证指数', ..., 12, 108.6160000000062,
        127.62400000000531]], dtype=object)

In [31]:
#统计沪深300每个工作日的涨跌幅
pd.set_option('expand_frame_repr',False)
df['涨跌幅']=df['收盘价']/df['收盘价'].shift(1)-1

#计算工作日
df['星期']=df['日期'].dt.dayofweek
df['星期']+=1
#统计各个工作日的均值，涨跌幅等特征
result=df.groupby('星期')['涨跌幅'].describe()
tmp1=df.groupby('星期')['涨跌幅'].size()
tmp2=df[df['涨跌幅']>0].groupby('星期')['涨跌幅'].size()
result['胜率']=tmp2/tmp1
print(result.T)

星期              1           2           3           4           5
count  559.000000  578.000000  582.000000  576.000000  568.000000
mean    -0.001913   -0.000131    0.000627   -0.001144   -0.000870
std      0.018893    0.018982    0.019501    0.019890    0.024695
min     -0.068989   -0.066143   -0.078916   -0.068841   -0.080367
25%     -0.012628   -0.010876   -0.010444   -0.012559   -0.014502
50%     -0.001761    0.000447    0.000988   -0.001441   -0.002552
75%      0.008417    0.010457    0.011063    0.009916    0.012291
max      0.083527    0.094096    0.106899    0.102037    0.105317
胜率       0.447227    0.510381    0.529210    0.451389    0.453427


In [32]:
# 查看在牛熊不同状况下的周内效应
# 牛市中，周一和周五表现较好，周二和周四表现较差
# 熊市中，周二和周三表现较好，周一和周四表现较差
#计算工作日
df['星期']=df['日期'].dt.dayofweek
df['星期']+=1
# 插入均线计算以及判断上涨市和下跌市
df.reset_index(drop=True,inplace=True)
df.loc[(df['收盘价']>df['收盘价'].rolling(20,min_periods=1).mean()),'上涨市_mean']=True
df['上涨市_mean'].fillna(value=False,inplace=True)
# 选择上涨市还是下跌市
df=df[df['上涨市_mean']==True]

#统计各个工作日的均值，涨跌幅等特征
result=df.groupby('星期')['涨跌幅'].describe()
tmp1=df.groupby('星期')['涨跌幅'].size()
tmp2=df[df['涨跌幅']>0].groupby('星期')['涨跌幅'].size()
result['胜率']=tmp2/tmp1
print(result.T)

星期              1           2           3           4           5
count  217.000000  238.000000  242.000000  237.000000  232.000000
mean     0.005127    0.004848    0.006889    0.006021    0.011035
std      0.018992    0.019524    0.019100    0.020861    0.025411
min     -0.049751   -0.060953   -0.050163   -0.052100   -0.061338
25%     -0.005965   -0.005468   -0.002871   -0.004682   -0.003910
50%      0.003578    0.005412    0.005886    0.004694    0.008738
75%      0.014483    0.015488    0.015767    0.015543    0.023584
max      0.083527    0.094096    0.106899    0.102037    0.105317
胜率       0.617512    0.634454    0.673554    0.594937    0.676724
